Se va a utilizar el dataset de Alex Krizhevsky (2009) de imágnes de airplane, automobile, bird, cat, deer, dog, frog, horse, ship y truck, que consiste en 60000 imágenes RGB de 32x32 pixeles equilibradamente distribuidas en las dos clases.

## Preprocesamiento

In [2]:
import numpy as np

In [3]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

Vamos a reacomodar los tensores para que sean en los 3 rangos de la imagen original. Dejaremos 10000 para validación, 10000 para test y el restante (40000) para entrenamiento. 

Modificamos las escalas de colores (0 a 255) con valores en una escala que vaya de 0 a 1 y que sean entonces de tipo flotantes.

Finalmente, convertimos los labels de arrays one hot encoded.

In [33]:
dire = 'cifar-10-batches-py/'

batch_1_dict = unpickle(dire+'data_batch_1')
data1 = batch_1_dict[b'data'].reshape(10000, 3, 32, 32)
data1 = np.transpose(data1, (0, 2, 3, 1))
labels1 = batch_1_dict[b'labels']
labels1 = np.asarray(labels1)

batch_2_dict = unpickle(dire+'data_batch_2')
data2 = batch_2_dict[b'data'].reshape(10000, 3, 32, 32)
data2 = np.transpose(data2, (0, 2, 3, 1))
labels2 = batch_2_dict[b'labels']
labels2 = np.asarray(labels2)

batch_3_dict = unpickle(dire+'data_batch_3')
data3 = batch_3_dict[b'data'].reshape(10000, 3, 32, 32)
data3 = np.transpose(data3, (0, 2, 3, 1))
labels3 = batch_3_dict[b'labels']
labels3 = np.asarray(labels3)

batch_4_dict = unpickle(dire+'data_batch_4')
data4 = batch_4_dict[b'data'].reshape(10000, 3, 32, 32)
data4 = np.transpose(data4, (0, 2, 3, 1))
labels4 = batch_4_dict[b'labels']
labels4 = np.asarray(labels4)

batch_5_dict = unpickle(dire+'data_batch_5')
data_validation = batch_5_dict[b'data'].reshape(10000, 3, 32, 32)
data_validation = np.transpose(data_validation, (0, 2, 3, 1))
labels_validation = batch_5_dict[b'labels']
labels_validation = np.asarray(labels_validation)

batch_test = unpickle(dire+'test_batch')
data_test = batch_test[b'data'].reshape(10000, 3, 32, 32)
data_test = np.transpose(data_test, (0, 2, 3, 1))
labels_test = batch_test[b'labels']
labels_test = np.asarray(labels_test)

data_train = np.concatenate((data1,data2,data3,data4)).astype('float32') / 255 
data_validation = data_validation.astype('float32') / 255 
data_test = data_test.astype('float32') / 255 
labels_train = np.concatenate((labels1,labels2,labels3,labels4))
labels_train = to_categorical(labels_train)
labels_validation = to_categorical(labels_validation)
labels_test = to_categorical(labels_test)
print(labels_train.shape, labels_validation, labels_test)
print(data_train.shape, data_validation.shape, data_test.shape)

NameError: name 'to_categorical' is not defined

## Definición del modelo

Vamos a definir un modelo:
* que recibe un tensor con las instancias de imágen con tres ejes con la dimensionalidad (32, 32, 3), 
* con 3 capas convolucionales 2D, todas con filtros 3x3 y función de activación RELU; la primera con 32 filtros, la segunda con 64 filtros, y la tercera con 128 filtros.
* con 3 capas intermedias de MAX pooling que toman las salidas de las capas convolucionales.
* con una capa Flatten que permite cambiar la representación de las salidas de las capas convolucionales a un tensor de un solo eje
* con dos capas Dense, fully connected, la primera con 512 neuronas y RELU, la segunda, la capa de salida, con 10 neuronas que utilizan una función softmax para clasificar las instancias en diez clases.
* una capa de dropout entre las capas densas con probabilidad de 30%.

In [1]:
import keras

C:\Users\bryan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation='softmax'))

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 4, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 2, 2, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 512)               0         
__________

## Entrenamiento del modelo

El modelo final tiene en total 356.417 parámetros. Lo compilamos utilizando un optimizador **rmsprop** y una función de activación **categorical_cross_entropy**, y lo entrenamos con 5 épocas con el set de entrenamiento ya procesado.

In [8]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_data=data_validation)

Train on 54000 samples, validate on 6000 samples
Epoch 1/5
54000/54000 [==============================] - 68s 1ms/step - loss: 0.2418 - acc: 0.9256 - val_loss: 0.0603 - val_acc: 0.9813
Epoch 2/5
54000/54000 [==============================] - 70s 1ms/step - loss: 0.0684 - acc: 0.9800 - val_loss: 0.0518 - val_acc: 0.9868
Epoch 3/5
54000/54000 [==============================] - 69s 1ms/step - loss: 0.0477 - acc: 0.9860 - val_loss: 0.0370 - val_acc: 0.9908
Epoch 4/5
54000/54000 [==============================] - 69s 1ms/step - loss: 0.0378 - acc: 0.9889 - val_loss: 0.0428 - val_acc: 0.9893
Epoch 5/5
54000/54000 [==============================] - 70s 1ms/step - loss: 0.0313 - acc: 0.9910 - val_loss: 0.0356 - val_acc: 0.9907
